# Broadcasting from the Foundations

In [1]:
from collections.abc import Iterable
from functools import wraps
from itertools import zip_longest, product
from math import log, exp

from fastcore.all import *

## The Goal

The goal of broadcasting is to be memory efficient. The key idea is that instead of storing the data in a nested list of lists, we store it in a shallow list plus a shape tuple and a strides tuple.

## The Rules

We are allow to use only plain python plus the standard library. We do use `fastcore` for testing and to split class methods into different cells of the notebook.

## Functions

### Flatten

We want to use as an input a nested list of lists, but we need a function to flatten this list before storing it into a class property.

In [2]:
def flatten(x):
    for item in x:
        if isinstance(item, Iterable):
            yield from flatten(item)
        else:
            yield item

In [3]:
l1 = []
l2 = [0]
l3 = [0,1]
l4 = [[0,1], [2,3]]
l5 = [[0,1], [2,3], [4,5]]
l6 = [[[0,1], [2,3], [4,5]], [[0,1], [2,3], [4,5]], [[0,1], [2,3], [4,5]], [[0,1], [2,3], [4,5]]]

In [4]:
test_eq(list(flatten(l6)), [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5])

### Shape

We also need a function to get the shape from a list of lists.

In [5]:
def get_shape(x):
    if isinstance(x, Iterable):
        s = (len(x),)
        if len(x)>0 and isinstance(x[0], Iterable):
            s += get_shape(x[0])
        return s

In [6]:
test_eq(get_shape(l1), (0,))
test_eq(get_shape([[]]), (1, 0))
test_eq(get_shape([[1]]), (1,1))
test_eq(get_shape([[1,2]]), (1,2))
test_eq(get_shape(l4), (2, 2))
test_eq(get_shape(l5), (3, 2))
test_eq(get_shape(l6), (4, 3, 2))

### Strides

Finally, we need a function to get the strides from the shape. A excelent explanation of what are the strides have been already written [elsewere](https://stackoverflow.com/a/53099870).

In [7]:
def get_strides_from_shape(shape):
    strides = (1,)
    for x in shape[:0:-1]: strides = (strides[0]*x,) + strides
    return strides

In [8]:
test_eq(get_strides_from_shape((2,2,4)), (8, 4, 1))

## The Matrix Class

Now we can write the `Matrix` class with a `__getitme__`, `__setitem__` and a `__repr__` methods.

In [9]:
class Matrix:
    
    def __init__(self, values=None, shape=None, strides=None):
        if values:
            self.values = list(flatten(values))
        else:
            self.values = [None] * round(exp(sum(map(log, shape))))
        if shape:
            self.shape = shape
        else:
            self.shape = get_shape(values)
        if strides:
            self.strides = strides
        else:
            self.build_strides_from_shape()
    
    def build_strides_from_shape(self):
        self.strides = get_strides_from_shape(self.shape)
        return self

    def __getitem__(self, idxs):
        i = sum(map(lambda x: x[0]*x[1], zip(idxs, self.strides)))
        return self.values[i]

    def __setitem__(self, idxs, val):
        i = sum(map(lambda x: x[0]*x[1], zip(idxs, self.strides)))
        self.values[i] = val

    def __repr__(self):
        if len(self.shape)==1:
            return str(self.values)
        elif len(self.shape)==2:
            s = ''
            for i in range(self.shape[0]):
                s += ', '.join([f'{self[i, j]}' for j in range(self.shape[1])])
                s += '\n'
            return s
        else:
            raise Exception('Not Implemented')

    def flatten_iter(self):
        flattened_indices = product(*map(range, self.shape))
        for index in flattened_indices:
            yield self[index]

In [10]:
m = Matrix([[0,1], [2,3]])
m

0, 1
2, 3

In [11]:
test_eq(m[1, 0], 2)

In [12]:
m[1, 1] = -1
test_eq(m[1, 1], -1)
m

0, 1
2, -1

In [13]:
Matrix(shape=(2,3))

None, None, None
None, None, None

### The Broadcasting Rules

[Broadcasting docs](https://numpy.org/doc/stable/user/basics.broadcasting.html)
>When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimensions and works its way left. Two dimensions are compatible when
>
>    1. they are equal, or
>
>    2. one of them is 1
>
>If these conditions are not met, a ValueError: operands could not be broadcast together exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the size that is not 1 along each axis of the inputs.

To build this functionality we can write a method to reshape the matrix without changing the `values` property by changing the `shape` and `strides` property that control how the `values` are accessed by the `__getitem__` method.

In [14]:
@patch
def as_shape(self:Matrix, shape):
    if shape == self.shape: return Matrix(values=self.values, shape=self.shape, strides=self.strides)
    strides = ()
    new_shape = ()
    for sdim, sstride, odim in zip_longest(self.shape[::-1], self.strides[::-1], shape[::-1]):
        if odim==sdim or odim==1 or odim is None:
            strides = (sstride,) + strides
            new_shape = (max(sdim, odim), ) + new_shape
        elif sdim==1 or sdim is None:
            strides = (0, ) + strides
            new_shape = (odim, ) + new_shape
        else:
            raise ValueError('operands could not be broadcast together')
    new = Matrix(values=self.values, shape=new_shape, strides=strides)
    return new

In [15]:
m1 = Matrix([[0, 1, 2]])

In [16]:
m1

0, 1, 2

In [17]:
m1.shape

(1, 3)

In [18]:
m1.as_shape((2, 3))

0, 1, 2
0, 1, 2

In [19]:
test_eq(m1.as_shape((2, 3)).strides, (0, 1))

In [20]:
test_eq(m1.as_shape((2, 3)).shape, (2, 3))

In [21]:
test_fail(lambda: m1.as_shape((2, 4)), msg='operands could not be broadcast together')

In [22]:
m1 = Matrix([0, 1, 2])

In [23]:
m2 = Matrix([[0,1, 2], [3,4,5]])

### The Broadcast Decorator

We can write a `broadcast` decorator that, if needed, reshapes the matrices that we want to add together before the operation takes place.

In [24]:
def broadcast(fun):
    @wraps(fun)
    def _inner(a, b):
        reshaped_a = a.as_shape(b.shape)
        reshaped_b = b.as_shape(a.shape)
        return fun(reshaped_a, reshaped_b)
    return _inner

### Broadcasting `__add__`

We can test the `boradcast` decorator on the `__add__` method.

In [25]:
@patch
@broadcast
def __add__(self:Matrix, other):
    result = Matrix(shape=self.shape)
    for i, (a, b) in enumerate(zip(self.flatten_iter(), other.flatten_iter())):
        result.values[i] = a + b
    return result 

In [26]:
m1 = Matrix([[0, 1, 2]])
m2 = Matrix([[0,1, 2], [3,4,5]])
m = Matrix([[0,1], [2,3]])

In [27]:
m

0, 1
2, 3

In [28]:
m+m

0, 2
4, 6

In [29]:
m1 = Matrix([[0,1, 2]])
m1

0, 1, 2

In [30]:
m2 = Matrix([[0,1, 2], [3,4,5]])
m2

0, 1, 2
3, 4, 5

In [31]:
m2+m1.as_shape((2, 3))

0, 2, 4
3, 5, 7

In [32]:
m1.as_shape((2, 3))+m2

0, 2, 4
3, 5, 7

In [33]:
m1+m2

0, 2, 4
3, 5, 7

In [34]:
m2+m1

0, 2, 4
3, 5, 7

In [35]:
m3 = Matrix([[0,1,2, 3], [4,5,6,7]])

In [36]:
m3

0, 1, 2, 3
4, 5, 6, 7

In [37]:
test_fail(lambda: m2+m3, msg='operands could not be broadcast together') 

In [38]:
test_fail(lambda: m3+m2, msg='operands could not be broadcast together') 